In [ ]:
import numpy as np
import torch
import torch.nn.functional as F

In [ ]:
# original image dimensions
# C * H * W
batch = 6
orig_img = np.arange(125*batch).reshape((batch, 5,5,5))
print(orig_img.shape)
print(orig_img)
batch, C, H, W = orig_img.shape

In [ ]:
# original kernels

F1 = 3  # Height of kernels
F2 = 2  # Widht of kernels
n_f = 5   # Number of kernels
orig_kernels = np.random.randn(n_f, C, F1, F2)
print(orig_kernels.shape)
print(orig_kernels)
n_f, C, F1, F2 = orig_kernels.shape

In [ ]:
strides = (2,1)
S1, S2 = strides

In [126]:
# Output feature image height and width(Leaving out padding for now)
H_ = np.int(np.floor((H - F1) / S1) + 1)
W_ = np.int(np.floor((W - F2) / S2) + 1)

In [ ]:
print(H_,W_)

In [ ]:
# https://stackoverflow.com/a/40840048
# Refer above link for awesome answer on im2col

def im2col(image,kernel_shape,strides=(1,1)):
    A = image
    B = kernel_shape
    skip = strides

    # Parameters 
    batch, D,M,N = A.shape
    col_extent = N - B[1] + 1
    row_extent = M - B[0] + 1

    # Get batch block indices
    batch_idx = np.arange(batch)[:, None, None] * D * M * N

    # Get Starting block indices
    start_idx = np.arange(B[0])[None, :,None]*N + np.arange(B[1])

    # Generate Depth indeces
    didx=M*N*np.arange(D)
    start_idx=(didx[None, :, None]+start_idx.ravel()).reshape((-1,B[0],B[1]))

    # Get offsetted indices across the height and width of input array
    offset_idx = np.arange(row_extent)[None, :, None]*N + np.arange(col_extent)

    # Get all actual indices & index into input array for final output
    act_idx = (batch_idx + 
        start_idx.ravel()[None, :, None] + 
        offset_idx[:,::skip[0],::skip[1]].ravel())

    out = np.take (A, act_idx)
    
    return out

In [ ]:

im2col_img = im2col(orig_img,(F1,F2),strides)
print(im2col_img)

In [ ]:
img_stacked = np.hstack((im2col_img))
img_stacked

In [ ]:
kernels = orig_kernels.reshape(n_f,-1)
print(kernels.shape)
print(kernels)

In [ ]:
final_img = np.matmul(kernels, img_stacked)
print(final_img)
print(final_img.shape)

In [ ]:
final_img = np.split(np.array(final_img), batch, axis=1)
print(final_img)

In [ ]:
final_img = np.float32(np.array(final_img).reshape(batch,n_f,H_,W_))
print(final_img)
print(final_img.shape)

In [ ]:
F.conv2d(torch.Tensor(orig_img), torch.Tensor(orig_kernels),stride=(2,1))